In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import cv2

import optuna
import wandb
import gc

2025-12-12 02:56:39.862011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-12 02:56:39.862034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-12 02:56:39.862842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-12 02:56:39.867124: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else: 
    print("No GPU detected")

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow is using the GPU 
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-12-12 02:56:47.128070: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-12 02:56:47.135999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-12 02:56:47.137712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Define LHCO column names (standard format, but can vary!)
columns = ["event", "typ", "eta", "phi", "pt", "jmas", "ntrk", "btag", "had_em", "dum1", "dum2"]

# Read file while ignoring comments (#)
df = pd.read_csv("/tf/Higgs-Boson-LHC-Collision-Detector/gg-HF.lhco", delim_whitespace = True, comment="#", names=columns)

df.head()

/tmp/ipykernel_1000/4090777018.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("/tf/Higgs-Boson-LHC-Collision-Detector/gg-HF.lhco", delim_whitespace = True, comment="#", names=columns)


,event,typ,eta,phi,pt,jmas,ntrk,btag,had_em,dum1,dum2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,0.514,-2.858,198.98,0.00,-1.0,0.0,0.00,0.0,0.0
3,2.0,2.0,-0.289,-2.435,92.99,0.11,1.0,5.0,94.00,0.0,0.0
4,3.0,4.0,1.997,0.118,265.01,39.93,15.0,0.0,2.23,0.0,0.0


In [7]:
# Count NaN values in each column
df.isna().sum()

# Or with isnull()
df.isnull().sum()

# Get percentage of NaN values per column
# (df.isna().sum() / len(df)) * 100

event           1
typ             1
eta             1
phi       1050001
pt        1050001
jmas      1050001
ntrk      1050001
btag      1050001
had_em    1050001
dum1      1050001
dum2      1050001
dtype: int64

In [9]:
ds = df.iloc[1:,:].reset_index().fillna(0)
ds = ds.iloc[:,1:]
ds.head(10)

,event,typ,eta,phi,pt,jmas,ntrk,btag,had_em,dum1,dum2
0,0.0,0.0,0.000,0.000,0.00,0.00,0.0,0.0,0.00,0.0,0.0
1,1.0,1.0,0.514,-2.858,198.98,0.00,-1.0,0.0,0.00,0.0,0.0
2,2.0,2.0,-0.289,-2.435,92.99,0.11,1.0,5.0,94.00,0.0,0.0
3,3.0,4.0,1.997,0.118,265.01,39.93,15.0,0.0,2.23,0.0,0.0
4,4.0,4.0,3.150,1.320,62.66,8.64,0.0,0.0,999.90,0.0,0.0
5,5.0,4.0,-3.357,2.711,22.55,3.86,0.0,0.0,1.29,0.0,0.0
6,6.0,6.0,0.000,-0.027,13.51,0.00,0.0,0.0,0.00,0.0,0.0
7,0.0,1.0,0.000,0.000,0.00,0.00,0.0,0.0,0.00,0.0,0.0
8,1.0,1.0,1.077,2.441,47.54,0.00,-1.0,0.0,0.00,0.0,0.0
9,2.0,2.0,-0.061,-0.882,59.66,0.11,1.0,0.0,60.00,0.0,0.0


In [10]:
ds["event"] = ds["event"].astype(int)
ds.head(10)

,event,typ,eta,phi,pt,jmas,ntrk,btag,had_em,dum1,dum2
0,0,0.0,0.000,0.000,0.00,0.00,0.0,0.0,0.00,0.0,0.0
1,1,1.0,0.514,-2.858,198.98,0.00,-1.0,0.0,0.00,0.0,0.0
2,2,2.0,-0.289,-2.435,92.99,0.11,1.0,5.0,94.00,0.0,0.0
3,3,4.0,1.997,0.118,265.01,39.93,15.0,0.0,2.23,0.0,0.0
4,4,4.0,3.150,1.320,62.66,8.64,0.0,0.0,999.90,0.0,0.0
5,5,4.0,-3.357,2.711,22.55,3.86,0.0,0.0,1.29,0.0,0.0
6,6,6.0,0.000,-0.027,13.51,0.00,0.0,0.0,0.00,0.0,0.0
7,0,1.0,0.000,0.000,0.00,0.00,0.0,0.0,0.00,0.0,0.0
8,1,1.0,1.077,2.441,47.54,0.00,-1.0,0.0,0.00,0.0,0.0
9,2,2.0,-0.061,-0.882,59.66,0.11,1.0,0.0,60.00,0.0,0.0


In [13]:
event_boundaries = (ds['event'] == 0).cumsum()
events = [group for _, group in ds.groupby(event_boundaries)]
print(f"Number of events: {len(events)}")

Number of events: 1050000


In [14]:
filtered_events = []

for event_id, event_df in df.groupby(event_boundaries):
    has_type1 = 1 in event_df['typ'].values
    has_type2 = 2 in event_df['typ'].values
    
    if has_type1 and has_type2:
        filtered_events.append((event_id, event_df))

In [15]:
print(f"\nTotal events: {event_boundaries.nunique()}")
print(f"Events with both type 1 and 2 particles: {len(filtered_events)}")


Total events: 1050000
Events with both type 1 and 2 particles: 442861
